# Part 1---Segmenting and Clustering Neighborhoods in Toronto

In [64]:
import pandas as pd

In [65]:
tables=pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
df=tables[0]
df.head()
df.shape

(180, 3)

### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [66]:
df=df[df["Borough"]!="Not assigned"]
df.shape

(103, 3)

### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [68]:
# there is no neighborhood as "Not assigned" after removing "Not assigned" Borough

list=df[df["Neighbourhood"] == 'Not assigned']

len(list)


0

### After data clean, there are 103 rows left in table.

In [70]:
df.shape

(103, 3)